In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from sentence_transformers import SentenceTransformer, util
import pyarrow as pa
import pyarrow.parquet as pq

from snowflake import connector as sf
import os

torch.manual_seed(42)

/home/twoon/.local/lib/python3.8/site-packages/snowflake/connector/options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (12.0.1), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [3]:
def load_snowflake_data(query):
    connection = sf.connect(
        user= 'twoon@moneylion.com', # replace with your moneylion email.
        account='moneylion.us-east-1',
        authenticator='externalbrowser',
        database='ANALYTICS_DB',
        warehouse='PROD_ANALYST_READ_WH',
        role='PROD_A_USR_TWOON_WRITER' # replace with your SF role 
    ) 

    connection.autocommit = True

    df = pd.read_sql_query(query, connection)

    connection.close()
    
    return df

In [28]:
def sentence_embeddings(dataset, batch_size, parquet_file_name):
    # set random state
    torch.manual_seed(42)
    
    # read dataset
    df_whole = dataset
    df_whole.columns = list(df_whole.columns.str.lower())
    # add a new column that shorten the description
    df_whole['short_desc'] = df_whole['description'].apply(lambda x: 
                                             x if len(x) <= 30 else 
                                             (x[:30] if x[30]==' ' else 
                                              (x[:x[30:].index(' ') + 30] if ' ' in x[30:] else x)
                                             )
                                            )
    
    # create empty parquet file
    # if not os.path.exists(parquet_file_name):
    df_empty = pd.DataFrame(columns=list(df_whole.columns.str.lower()))
    table_empty = pa.Table.from_pandas(df_empty)
    pq.write_table(table_empty, parquet_file_name)
    
    # define tokenizer and model
    model = SentenceTransformer('all-MiniLM-L6-v2')
    
    # define batch size
    batch_size = int(batch_size)
    
    while df_whole.shape[0] > 0:
        to_take = min(batch_size, df_whole.shape[0])
        df = df_whole[:to_take]
        df_whole = df_whole[to_take:]
    
        # Sentence Transformer - all-MiniLM-L6-v2
        ## embeddings
        df['emb'] = df.loc[:,'short_desc'].apply(lambda x: model.encode(x))
        ## define chargeback text
        text = 'ACH DISPUTE ' + 'Adjustment - Cr ' + 'Misc. Credit Adjustment on ' + 'ACH Claim# ' + 'Reversal'
        text_emb = model.encode(text)
        ## calculate cosine similarity
        df['similarity'] = df.loc[:,'emb'].apply(lambda x: util.cos_sim(x, text_emb))
        df['similarity'] = df.loc[:,'similarity'].astype('float32')
        ## make prediction ; 1 = chargeback/unauth , 0 = good transaction
        threshold = 0.3
        df['pred'] = df.loc[:,'similarity'].apply(lambda x: 1 if x>= threshold else 0)
        
        # read existing parquet file and convert it to dataframe
        table = pq.read_table(parquet_file_name).to_pandas()
        
        # concat the new dataframe and dataframe from parquet file
        df_concat = pd.concat([table, df])
        
        # store in parquet file
        table_new = pa.Table.from_pandas(df_concat)
        pq.write_table(table_new, parquet_file_name)

    n_pred_chb = df_concat[df_concat['pred']==1].pred.count()
    n_pred_not_chb = df_concat[df_concat['pred']==0].pred.count()

In [5]:
query = """with dc_txn_user as (
  select distinct user_id
  from ANALYTICS_DB.DBT_PAYMENTS.FCT_SUCCESSFUL_CARD_CHARGES
  where action = 'DC_FUNDING' 
  and created_at between dateadd('day', -60, current_date) and current_date
)

select
    bv_transaction_id,
    amount, 
    description,
    memo,
    categorization,
    transaction_date,
    created_on,
    updated_on,
    bank_name,
    provider,
    bank_status,
    merchant_name,
    bv_account_id,
    user_id,
    first_transaction_date,
    bank_created_on
from ANALYTICS_DB.DBT_BANKING.BASE_TRANSACTION_ENRICHED_V2
where user_id in (select user_id from dc_txn_user)
and transaction_date between dateadd('day',-180,current_date) and dateadd('day',-60,current_date)"""

df_main = load_snowflake_data(query)

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
We were unable to open a browser window for you, please open the following url manually then paste the URL you are redirected to into the terminal.
URL: https://sso.jumpcloud.com/saml2/snowflakeai?SAMLRequest=lZJbc9owEIX%2Fikd9tmUbpxANJkPjpvGU2wBh0r4JWxARWXK0Eib59ZW5dNKHZKZvGukcfbt7tn9zqIS3Zxq4kimKghB5TBaq5HKbooflnd9DHhgqSyqUZCl6ZYBuBn2glajJ0JonOWcvloHx3EcSSPuQIqslURQ4EEkrBsQUZDEcj0gchIQCMG0cDp0tJXDHejKmJhg3TRM0nUDpLY7DMMThNXaqVvIFvUPUnzNqrYwqlLhYDq6nDxARDpMW4RSOMDsbv3F5GsFnlPVJBOR%2BuZz5s%2BliibzhpbtbJcFWTC%2BY3vOCPcxHpwLAVTCeTr7%2FGuXTSWDBZxSMHwUgVbMR9JkVqqqtcf8G7oQ3rMRCbbmbVp6lqH7mZedeJXSbdx%2FX9fhlfej%2BnNlRBm%2B7xWq12xfb9Y%2FeW7Yxj3S%2BSArkrS7Zxm22OYBluWwTNe4qjDt%2B2PPjZBnGJLoiSTdIvl79Rl7mEuWSmqPzUjaACna2qguhbHmsro0ixn9LpxydVoMcOXrw%2Fw338Xv%2Fec8mbvR

Enter the URL the SSO URL redirected you to:  http://localhost:52945/?token=tVhXl5tM0v4rc_Re6owBgeLxeD-SCCIIgQi6IzRBIokGEX79Io1nXnvstb3f7l6hqq6u-HS1qj__o8vSpxuoYFLkLxPsEzp5ArlfBEkevUyOxvZ5NfnHl8_QzdJZuTkAWBY5BE_jphxuXrkvk6bKN4ULE7jJ3QzATe1vdFKWNrNP6Kasirrwi3TydU8Hg5dJXNflBkHatv3U4p-KKkJmKIohtizpfgwyd_JE383k9a91-69CmyaHJfCTMAHB5IkBsE5yt37EczcER0tZkYM-HVmfGvgMXFg_Y59gXrRh6l6AX2RlM-6JPo2_kBAESFpEST55EpiXiXASFBmTdowusAt6adlLbE6bB10lZifanpGCsqBO6IEbpfO39BjFy6S8JAHOF4QbCUvbK-Wr1y13-0Zi4HDWTfN88yOPWw1MWNvuQSf8cT-EDRByWLv3uGfoDH9GV88zwkBnG2yxwZefiOX6NHky34o1pmDytTSbx-bq27r8OnUuhKC652jytC2qzP1Npu-cJHgOH6KbMedJ3U--iE1W0mnRBJ-Rb7348jmAGz2JxiI01RtWAvgvyo4i6BoZZQKYRH9N3veCQMjD4kHSbl7kie-myfAoqwzquAieyDQqqqSOs3-JJwy9K34Gnf_sY0T-1wT53rU_VoQSbx4-Z0UF_qqg-wxjdzZffFV5ACGoxnMDno4H4WXy1x-j5rHbqNwc3nMLvyd_69l3uQP5DaRFCYJn-BbgV-_-XOFPcvblM_A3Qu6nDUxuQLlDo3R9AL_WFfh_ruppX4Ew6aQEjnAbG8HdPeRb_z6Qr-lgkmg80v9mrcZa_PVdhV61mG7agC-9ynqWX7a2kS9jK8_bQU3tlBYSvq_2iChjtV2GRjfwyeXl4dG3mx-M93q_kh8A-w6w1x2MfAmq8LiGJdvokKFO

/tmp/ipykernel_38737/2963586961.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


In [6]:
table_main = pa.Table.from_pandas(df_main)
pq.write_table(table_main, 'main.pq')

14631010

In [18]:
n = df_main.shape[0]
n

14631010

In [26]:
new_df = df_main.iloc[5500000:]

In [32]:
new_df.shape

(9131010, 17)

In [40]:
new_df.iloc[5499995:5500001]

,bv_transaction_id,amount,description,memo,categorization,transaction_date,created_on,updated_on,bank_name,provider,bank_status,merchant_name,bv_account_id,user_id,first_transaction_date,bank_created_on,short_desc
10999995,f49eb224-eb46-401c-be6e-26f932aafc6d,-8.82,Amazon,Shops Digital Purchase,"Shops, Digital Purchase",2023-05-19 00:00:00,2023-05-19 07:02:58.527,2023-08-22 20:33:53.316,Capital One,PLAID,COMPLETE,None,5f32f4ad-8560-413b-afb8-cd95019cf415,63180ccd7629ff7c8a2b8da2,2022-06-09 00:00:00,2022-09-07 03:23:26.035,Amazon
10999996,d404493c-249e-4b01-8955-77390cd3c375,-18.72,King Of Ro,Food and Drink Restaurants,"Food and Drink, Restaurants",2023-02-26 00:00:00,2023-02-26 21:51:20.554,2023-06-26 03:04:51.058,Chime Bank,PLAID,COMPLETE,None,83bca2c5-908f-4460-9a7d-412d61073deb,62e17982f716236c9bba58e7,2021-07-28 00:00:00,2022-07-27 18:01:04.005,King Of Ro
10999997,d1bfde5b-3496-46d2-a4fe-81c88cb5b450,0.32,Deposit Transfer From Share 0007 / CHANGE UP XFR,Transfer Credit,"Transfer, Credit",2023-03-22 00:00:00,2023-04-28 14:19:14.528,2023-04-28 14:20:04.657,Fortera Credit Union,PLAID,COMPLETE,None,aa395746-05ec-4d65-bfe5-13d8fb46fbab,5a919e2a4dc44100013d4d95,2022-04-29 00:00:00,2023-04-28 14:19:13.822,Deposit Transfer From Share 0007
10999998,41860c3a-ae9d-460c-be9e-79ed7a776188,-29.57,PURCHASE AUTHORIZED ON 05/11 CASEYS #3471 15 W...,Travel Gas Stations,"Travel, Gas Stations",2023-05-12 00:00:00,2023-05-13 13:50:40.363,2023-05-13 13:50:45.007,Wells Fargo,PLAID,COMPLETE,None,84a9f64a-cf41-4fbb-8d6b-915e5ae2d1d4,62d02b2c12f3f13ce873fa42,2021-07-16 00:00:00,2022-07-14 14:44:52.880,PURCHASE AUTHORIZED ON 05/11 CASEYS
10999999,32dfa4dc-9db7-4835-95dc-9c5319251935,5.66,Round Up Transfer,Payment,Payment,2023-04-27 07:00:15,2023-04-27 07:08:02.988,2023-08-14 07:01:33.239,MoneyLion,ML,COMPLETE,None,bdd8db23-f906-4f13-b421-5c82adf1af79,639131bb95da2527dcfe4f27,2022-12-12 08:32:37,2022-12-08 16:19:17.867,Round Up Transfer
11000000,095233e4-c548-4f11-9f72-9869264b3a13,-3.00,Out of Network ATM fee 7559 MIDLOTHIAN TURNPIK...,Bank Fees ATM,"Bank Fees, ATM",2023-05-21 00:00:00,2023-05-22 23:28:12.321,2023-05-22 23:28:14.622,Varo Bank,PLAID,REMOVED,None,a3b89d01-97fb-4286-b06b-15ca6f92cccd,63622f2453373f5a2376a42b,2022-10-05 00:00:00,2022-11-02 09:00:27.089,Out of Network ATM fee 7559 MIDLOTHIAN


In [35]:
new_df_3rd_batch = df_main.iloc[11000000:]

In [42]:
new_df_3rd_batch.shape

(3631010, 17)

In [43]:
sentence_embeddings(new_df_3rd_batch,2e5,'sentence_embeddings_pt3.pq')

/tmp/ipykernel_38737/1059276214.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_whole['short_desc'] = df_whole['description'].apply(lambda x:
/tmp/ipykernel_38737/1059276214.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['emb'] = df.loc[:,'short_desc'].apply(lambda x: model.encode(x))
/tmp/ipykernel_38737/1059276214.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [29]:
sentence_embeddings(new_df,2e5,'sentence_embeddings_pt2.pq')

/tmp/ipykernel_38737/1059276214.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_whole['short_desc'] = df_whole['description'].apply(lambda x:
/tmp/ipykernel_38737/1059276214.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['emb'] = df.loc[:,'short_desc'].apply(lambda x: model.encode(x))
/tmp/ipykernel_38737/1059276214.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

OSError: List index overflow.

In [21]:
table

NameError: name 'table' is not defined

In [8]:
df_extra = pq.read_table('sentence_embeddings.parquet').to_pandas()

In [9]:
df_extra.shape

(4400000, 20)

In [15]:
df_extra[df_extra['pred']==0].pred.count() / df_extra.pred.count()

0.9521265909090909

In [10]:
df_main.shape

(14631010, 16)

In [45]:
table_main = pa.Table.from_pandas(df_main)
pq.write_table(table_main, 'dataset_before_embeddings.pq')

In [47]:
df_main.bv_transaction_id.nunique()

14631010

In [48]:
test = pd.concat([new_df, new_df_3rd_batch])

In [49]:
test.shape

(12762020, 17)

In [52]:
new_df.bv_transaction_id.nunique()

9131010

In [53]:
new_df.shape

(9131010, 17)

In [58]:
df_main.bv_transaction_id.nunique()

14631010

In [59]:
1463-1455

8

In [60]:
df_main[df_main['bv_transaction_id']=='2a96d037-b29c-4969-8ad5-e8eeea35ff92']

,bv_transaction_id,amount,description,memo,categorization,transaction_date,created_on,updated_on,bank_name,provider,bank_status,merchant_name,bv_account_id,user_id,first_transaction_date,bank_created_on,short_desc
9391482,2a96d037-b29c-4969-8ad5-e8eeea35ff92,-10.8,Withdrawal POS #000000465918 GAS EXPRESS 209 X...,Travel Gas Stations,"Travel, Gas Stations",2023-03-17,2023-08-03 16:00:58.814,2023-08-03 16:01:23.166,Baxter Credit Union - BCU,PLAID,COMPLETE,GAS EXPRESS,81d6486e-2b7e-4e18-89ea-1932ae5983d8,64cbce8bd66cf55f469b9792,2022-08-04,2023-08-03 16:00:58.587,Withdrawal POS #000000465918 GAS


In [64]:
a = 'analytics_db.payments_seed.sentence_transfromer_pred__2023_08_28'
a = a.lower()

In [65]:
b = 'ANALYTICS_DB.PAYMENTS_SEED.SENTENCE_TRANSFORMER_PRED__2023_08_28'
b = b.lower()

In [69]:
a==b

False

In [68]:
print(a)
print(b)

analytics_db.payments_seed.sentence_transfromer_pred__2023_08_28
analytics_db.payments_seed.sentence_transformer_pred__2023_08_28
